In [45]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Embedding, MultiHeadAttention, Dense, LSTM, Input

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder



In [46]:
#ATS System
df = pd.read_csv('UpdatedResumeDataSet.csv')
df.head()
df.info()
max_len = 300
resumes = df['Resume']
categories = df['Category']
Tokenizer = Tokenizer()
Tokenizer.fit_on_texts(resumes)
sequences = Tokenizer.texts_to_sequences(df['Resume'].values)

X = pad_sequences(sequences, maxlen=max_len, padding='post')
y = pd.get_dummies(df['Category']).values

label_encoder = LabelEncoder()  # Use LabelEncoder for string categories
categories_encoded = label_encoder.fit_transform(categories)
categories_encoded = to_categorical(categories_encoded)  # One-hot encode for multi-class classification


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  96 non-null     object
 1   Resume    96 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB


In [50]:
embeddings_dim = 100
embeddings_index = {}
with open('glove.6B.100d.txt', encoding = 'utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(Tokenizer.word_index) + 1, embeddings_dim))
for word, i in Tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
input_layer = Input(shape=(max_len,))

# Embedding layer
embedding_layer = Embedding(len(Tokenizer.word_index) + 1, embeddings_dim, input_length=max_len, weights=[embedding_matrix], trainable=False)(input_layer)

# LSTM layer
lstm_layer = LSTM(units=128, return_sequences=True)(embedding_layer)

# MultiHeadAttention layer
attention_layer = MultiHeadAttention(num_heads=2, key_dim=2)(lstm_layer, lstm_layer)

# Second LSTM layer
lstm_layer_2 = LSTM(units=64)(attention_layer)

# Output layer
output_layer = Dense(7, activation='softmax')(lstm_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [51]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64, callbacks=[EarlyStopping(patience=2)])
loss,auc = model.evaluate(X_test, y_test)
print(f"Loss: {loss}")

new_resume = "I am a software engineer with 5 years of experience in Java, Python, and C++. I have a Bachelor's degree in Computer Science from MIT."
new_resume = Tokenizer.texts_to_sequences([new_resume])
new_resume = pad_sequences(new_resume, maxlen=max_len)
prediction = model.predict(new_resume)
print(f"Predicted category: {categories[np.argmax(prediction)]}")


Epoch 1/10


2/2 [==============================] - 4s 782ms/step - loss: 1.9469 - accuracy: 0.0395 - val_loss: 1.9433 - val_accuracy: 0.1000
Epoch 2/10
2/2 [==============================] - 1s 264ms/step - loss: 1.9379 - accuracy: 0.1974 - val_loss: 1.9543 - val_accuracy: 0.1000
Epoch 3/10
1/1 [==============================] - 0s 88ms/step - loss: 1.9690 - accuracy: 0.1000
Loss: 1.9690393209457397
1/1 [==============================] - 0s 496ms/step
Predicted category: DevOps


In [12]:
import tensorflow as tf
print(tf.__version__)

2.15.0
